# 1. Introduction

   
Udemy is an online platform for learning and teaching. It has more than 213000 courses spanning over 75 languages. Courses on Udemy are useful for improving skills that are required for jobs. In this notebook, we will perform data analysis of Udemy courses and comments. There are two datasets provided:

* Course dataset and
* Comment dataset. 

The course dataset contains information about 210k courses offered on Udemy. The important information includes the title of a course, price, average rating, number of subscribers, number of lectures, category, subcategory, number of reviews, language, etc. The comment dataset contains nearly 9 million user comments for courses on Udemy.


# 2. Setup

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
#import langdetect
import plotly.express as px
import matplotlib.ticker as ticker
import ast

plotly.offline.init_notebook_mode (connected = True)

class color:
    BOLD = '\033[1m'
    END = '\033[0m'

# Load course dataset
courses = pd.read_csv("/kaggle/input/udemy-courses/Course_info.csv")

# Information about the course dataset
print(courses.info())

In [ ]:
# Statistics for numerical columns
courses.describe().transpose()

In [ ]:
courses.head()

# 3. Course Dataset - EDA

## 3.1 Missing Values

In [ ]:
# Checking for null values in the feature variables
print(color.BOLD + 'Missing values - Count: \n' + color.END)
print(courses.isnull().sum())
print('--'*20)

print(color.BOLD + 'Missing values - Percentage: \n' + color.END)
print(round(courses.isnull().mean() * 100, 2))

Out of 210k courses, only 0.74% of data are missing.

## 3.2 Price distribution

In [ ]:
# Distribution of price of udemy courses
fig, ax = plt.subplots(figsize=(10,6), dpi=100)
sns.histplot(data=courses, x='price', bins=75)
plt.xlabel("Price ($)")
plt.title("Distribution of the price of Udemy courses");

In [ ]:
# Box plot showing the distribution of price across category
plt.figure(figsize=(8,5), dpi=100)
sns.boxplot(data=courses, x='price', y='category')

The above plots show the distribution of the `price` of Udemy courses as well as the category-wise distribution (boxplot). Most of the courses are priced between 0 and $\$$200. Not all the courses are in the paid category. About 21738 courses are offered at no cost whereas 187996 courses have to be bought at a price. For the paid category, nearly 20\% of the courses are sold at $\$$19.99, and about 8.34\% of the courses are sold at $\$$199.99. The share of free and paid courses is visualized in the seaborn count plot below. Overall, around 10\% of the courses are available for free and the remaining courses (89.64\%) have to be bought.

In [ ]:
# Free or Paid Courses - Countplot
fig, ax = plt.subplots(figsize=(7,5), dpi=100)
ax = sns.countplot(data=courses, x='is_paid', palette='magma_r')
ax.set_xticklabels(labels=['Free', 'Paid'])
ax.set_xlabel("Free/Paid courses")
ax.set_ylabel("Number of courses")
ax.set_title("Share of Free and Paid Courses on Udemy")
percentage = round(courses['is_paid'].value_counts() * 100 /len(courses), 2)
patches = ax.patches
for i in range(len(patches)):
   x = patches[i].get_x() + patches[i].get_width()/2
   y = patches[i].get_height()+.05
   ax.annotate('{:.2f}%'.format(percentage[i]), (x, y), ha='center')

## 3.3 Course Languages in Udemy

In [ ]:
# Count plot for the number of courses in different langauges
plt.figure(figsize=(16, 8), dpi=120)
sns.countplot(data=courses, x='language', order=courses['language'].value_counts().index, palette='magma')
plt.xlabel("")
plt.ylabel("Number of courses in every language")
plt.title("Udemy Courses offered in different languages")
plt.xticks(rotation=90);

Based on the course dataset, there are 79 `languages` in which Udemy courses are offered. English (59%), Portugues (8.8%), and Spanish (8.3%) are the top three languages on Udemy. The top 15 languages in which the courses are offered are visualized in the pie chart shown below.

In [ ]:
# Top 15 languages
share_of_languages_top_15 = round(courses['language'].value_counts() * 100 / len(courses), 2)[:15]

# Pie chart for top 15 languages visualization
fig = px.pie(share_of_languages_top_15, values=share_of_languages_top_15.values, 
             names=share_of_languages_top_15.index, hole=0.25,
            )
fig.update_traces(textposition='inside', texttemplate = '%{label}<br>%{value}%', rotation=150)
fig.update_layout(title_text='Top 15 languages in which courses are offered', title_x=0.5, showlegend=False)
fig.show()

## 3.4 Courses under different categories

### 3.4.1 Number of courses in each category

In [ ]:
plt.figure(figsize=(10, 6), dpi=100)
sns.countplot(data=courses, y='category', order=courses['category'].value_counts().sort_values(ascending=False).index, palette='magma')
plt.xlabel("Number of courses")
plt.ylabel("")
plt.title("Number of courses offered in each category");

### 3.4.2 Number of subscribers in each category

In [ ]:
category_subs = courses.groupby('category')['num_subscribers'].agg('sum').sort_values(ascending=False)

# Bar plot to diplay number of subscribers across categories
fig, ax = plt.subplots(figsize=(10, 6), dpi=100)
sns.barplot(y=category_subs.index, x=category_subs.values, palette='magma', ci=0)
ax.set_xlabel("Number of subscribers")
ax.set_ylabel("")
ax.xaxis.set_major_formatter(ticker.EngFormatter())
ax.set_title("Number of subscribers per category");

The courses are covered under 13 `categories` and then these categories are further divided into 130 `subcategories`. There are 3818 unique `topics` under which different courses are offered. The category `Development` contains the most number of courses (31643), followed by `IT & Software` (30479) and `Teaching & Academics` (26293). The category `Development`  also topped the list for having the highest number of subscribers (~213 Million), followed by `IT & Software (106 Million)` and `Business` (70 Million).

### 3.4.3 Top 5 courses with highest number of reviews

In [ ]:
my_cols = ['title', 'instructor_name', 'price', 'num_reviews', 'num_subscribers', 'num_comments', 'content_length_min']

def top5_courses(df, col):
    top5_df = df.nlargest(5, col).reset_index(drop=True)
    return top5_df[my_cols]

In [ ]:
top5_courses(courses, 'num_reviews')

### 3.4.4 Top 5 courses with highest number of subscribers

In [ ]:
top5_courses(courses, 'num_subscribers')

### 3.4.5 Top 5 courses with highest number of comments

In [ ]:
top5_courses(courses, 'num_comments')

### 3.4.6 Top 5 courses with highest content length

In [ ]:
top5_courses(courses, 'content_length_min')

### 3.4.7 Most expensive Udemy courses

In [ ]:
top5_courses(courses, 'price')

### 3.4.8 Subcategory Level Data Visualization

In [ ]:
# At the subcategory level, the number of courses and subscribers are determined.
courses_subcat_df = courses.groupby(['category'])['subcategory'].value_counts().to_frame()
courses_subcat_df.columns = ['number of courses']
courses_subcat_df = courses_subcat_df.reset_index()

subscribers_subcat_df = courses.groupby(['category', 'subcategory'])['num_subscribers'].sum().to_frame()
subscribers_subcat_df.columns = ['number of subscribers']
subscribers_subcat_df = subscribers_subcat_df.reset_index()

# Sunburst chart for number of courses under each subcategory for different categories
fig = px.sunburst(courses_subcat_df, path=['category', 'subcategory'], 
                  values='number of courses', color='category',
                  color_continuous_scale='RdBu', width=600, height=600)
fig.update_layout(title_text='Number of courses offered in different subcategories', title_x=0.5)
fig.show()

# Sunburst chart for number of subscribers under each subcategory for different categories
fig = px.sunburst(subscribers_subcat_df, path=['category', 'subcategory'], values='number of subscribers',
                  color_continuous_scale='RdBu', width=600, height=600)
fig.update_layout(title_text='Number of subscribers in different subcategories', title_x=0.5)
fig.show()

We already know the `Development` category has the highest subscriber count (about 213 Million). Under the `Development` category, subcategories such as `Web Development` and `Programming Languages` have 76.6 Million and 58.5 Million subscribers respectively. The sunburst charts display the number of courses and subscribers at each subcategory level for different categories. The category `Music` has the least number of subscribers (about 8.5 Million) with the `Instruments` having the highest (about 3.9 Million) at the subcategory level.

## 3.5 Instructor Earnings

In [ ]:
courses['earnings_$'] =  courses['price'] * courses['num_subscribers']

instructor_earnings = pd.DataFrame(courses.groupby('instructor_name')['earnings_$'].sum().round(2))
instructor_earnings = instructor_earnings.sort_values(by='earnings_$', ascending=False)
instructor_earnings['earnings_$'] = instructor_earnings['earnings_$'].apply(lambda x: "{:,}".format(x))

# Top 10 Instructors based on their earnings
print(color.BOLD + 'Top 10 instructors with their earnings in $:' + color.END)
instructor_earnings.head(10)

There are 72731 `instructors` on the Udemy platform. The `total income` of the instructors is calculated by multiplying the 'price' and 'num_subscribers' columns. The above data frame shows the top 10 instructors with their income. Note that for the `earnings` calculation, discounts and coupons offered are not considered as their information is not available.

## 3.6 Year-wise data visualization

In [ ]:
# published_time column is cleaned and then converted into a datatime object
courses['published_time'] = pd.to_datetime(courses['published_time'].str[0:10])
courses['published_year'] = courses['published_time'].dt.year
courses['published_month'] = courses['published_time'].dt.month_name()

# Group the year column and perform operations for data visualization
courses_year_count = courses['published_year'].value_counts()

courses_num_subs = courses.groupby('published_year')['num_subscribers'].sum()

courses_num_reviews = courses.groupby('published_year')['num_reviews'].sum()

courses_num_comments = courses.groupby('published_year')['num_comments'].sum()

courses_num_lectures = courses.groupby('published_year')['num_lectures'].sum()

group_courses_year = pd.pivot_table(courses, index='published_year', 
                               values=['id', 'num_subscribers', 'num_reviews', 'num_comments', 'num_lectures'], 
                               aggfunc={'id': len, 'num_subscribers': np.sum, 'num_reviews':np.sum, 
                                        'num_comments':np.sum, 'num_lectures':np.sum})

print(color.BOLD + 'Year-wise count and sum values for different features:' + color.END)
group_courses_year

In [ ]:
# Visualization of year-wise feature data
dict_cols = dict({'id': 'Number of courses published', 'num_subscribers': 'Total subscribers', 
                  'num_comments': 'Total comments', 'num_reviews': 'Total reviews',
                 'num_lectures': 'Total lectures'})

for key, val in dict_cols.items():
    df = group_courses_year
    fig, ax = plt.subplots(figsize=(8,5), dpi=100)
    sns.barplot(data=df, x=df.index, y=df[key], palette='magma')
    ax.set_xlabel("Year")
    ax.set_ylabel(val)
    ax.set_title(f'{val} per year')
    ax.yaxis.set_major_formatter(ticker.EngFormatter());

***

# 4. Comments Dataset - EDA


The comments dataset contains six feature columns and nearly 9.5 million row values. The `comment` column and `rate` column are important for data analysis.


In [ ]:
# Load comments dataset
comments = pd.read_csv("/kaggle/input/udemy-courses/Comments.csv")

# Information about comments dataset
comments.info(show_counts=True)

In [ ]:
comments.head()

## 4.1 Missing Values

In [ ]:
# Display the features with missing values 
print(color.BOLD + 'Number of missing values:' + color.END)
print(comments.isnull().sum())
print("---"*20)
print(color.BOLD + 'Percentage of missing values:' + color.END)
print(round(comments.isnull().mean() * 100, 2))

In [ ]:
# Dropping rows with any column having NaN.
comments = comments.dropna(axis=0).reset_index(drop=True)

# Mapping the course title from the courses dataframe to comments dataframe
dict_id = dict(zip(courses['id'], courses['title']))
comments['course_title'] = comments['course_id'].map(dict_id)

The `comments dataset` and `courses dataset` both have a common column (`course_id`). To identify the course title against which the reviewers made comments, a new column named `course_title` for the comments data frame is created by mapping the `course_id` from the courses dataset.

## 4.2 Creating new column named "review" based on course ratings


By going through a sample of the `comments` dataframe, the `rate` column and `comments` column are related. Now we create a new label based on the `rate` column in the `comments` dataframe. The rate value ranges between 0.5 and 5.0. The new label, `review`, is positive if the rate value is greater than 3.0 and neutral if the rate is equal to 3.0. Otherwise, the review is negative. This new variable can be used as a target label for classification purposes.

In [ ]:
# New column 'review' based on 'rate' column.
comments['review'] = pd.cut(comments['rate'], bins=[0, 2.5, 3.0, 5.0], labels=['Negative', 'Neutral', 'Positive'])
#comments['review'] = comments['rate'].apply(lambda x: 'Positive' if x >= 3.0 else 'Negative')
comments.head()

For the classification problem, we will be considering the `comment` column and the `review` target label. The main goal of this section is to analyze the comments made by the users and then identify the language of the comments. For this purpose, we need to filter the `comment` column as it contains several special characters, symbols and emojis.

In [ ]:
# Using strip() method to remove the newline character from a string. 
# The strip() method will remove both trailing and leading newlines from the string. 
# It also removes any whitespaces on both sides of a string.
comments['comment'] = comments['comment'].str.replace("\n", " ")

# Remove special characters from the comment column
spl_ch = ['!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '=', '_', '+'
         '[', ']', '{', '}', '|', ':', '"', ';', "'", ',', '.', '/', '<', '>', '`', '~', '?']

for ch in spl_ch:
    #mask = comments['comment'].str
    #comments = comments.drop(comments[mask].index, axis=0)
    comments = comments[comments['comment'] != ch]
    
comments = comments.reset_index(drop=True)

# Keeping the dataframe without digits characters in 'comment' column
comments = comments[~comments['comment'].str.isdigit()].reset_index(drop=True)

## 4.3 Detecting the language of a comment

In the `comments` dataframe, most of the features are unique identifiers except the columns `date`, `comment`, and `review`. We can create a new column that stores the languages in which the comments are made. With the `langdetect` module, we can identify the language of a comment made by a student. The `langdetect` module is a direct port of Google’s language-detection library that supports 55 languages. 
    
Although the special characters and digits in the `comment` column are removed, there are rows with symbols, emojis, mix of multiple characters and symbols. 

We use the language detect library which checks for the language of a sentence. If the detect operation comes across any unknown symbol, we will store the language as 'no'. We can delete the rows with 'no' language later.

In [ ]:
# Since the dataset is really huge, it would take several hours for the 'langdetect' 
# to go through all the rows. To save time on kaggle, the DataFrame is already processed, 
# created 'comment_language' column and saved as a separate 'csv' file (udemy_comments_languages.csv). 
# So, I am attaching the code below for performing the language detection operation for reference purpose.
    
# Detecting the language of the comments made by the users

# langdet = []
    
# for idx, val in comments['comment'].iteritems():                                         
#     try:                                                          
#         lang=langdetect.detect(val)                                      
#     except:                                                       
#         lang='no'
#         print("This row throws error:", comments.iloc[idx])
#         print("----"*10)
#         print("\n")
#     langdet.append(lang)    

# langdet_df = pd.DataFrame(langdet, columns=['Language'])
# langdet_df.to_csv('udemy_comments_languages.csv', index=False)

In [ ]:
# Reading the result of the langdetect operation on the 'comment' column of the DataFrame.
langdet_df = pd.read_csv('/kaggle/input/udemy-courses-additional-files/udemy_comments_languages.csv')

# Sample data of the language detect dataframe file
langdet_df.sample(10)

In [ ]:
# The language column of the langdet_df DataFrame contains 'no' for unrecognized language.
langdet_df[langdet_df['Language'] == 'no'].sample(5)

In [ ]:
print(color.BOLD + 'Total number of unrecognized comments:' + color.END, len(langdet_df[langdet_df['Language'] == 'no']))
print(color.BOLD + 'Shape of the langdet_df DataFrame:' + color.END, langdet_df.shape)
print(color.BOLD + 'Shape of the comments DataFrame:' + color.END, comments.shape)

Note that there may be some comments misclassified as 'no' due to few word counts or special characters in between the words.

In [ ]:
# Copying the elements of the 'comments' dataframe to a new dataframe named 'comments_df'
comments_df = comments.copy()

# Storing the comment language in comments_df dataframe
comments_df['comment_language'] = langdet_df['Language'].copy()

# Sample dataframe showing the languages for the comment column
comments_df.sample(5, random_state=88)

In [ ]:
# Sample dataframe showing the detected hindi language for the comments
comments_df[comments_df['comment_language'] == 'hi'].sample(5, random_state=88)

In [ ]:
# Removing 'no' class of 'comment_language' column
comments_df = comments_df[~(comments_df['comment_language'] == 'no')].reset_index(drop=True)

print(color.BOLD + 'Shape of the comments_df DataFrame:' + color.END, comments_df.shape)

In [ ]:
# Share of positive and negative reviews
review_counts = comments_df['review'].value_counts()

perc_pos = str(np.round(comments_df['review'].value_counts()[0] * 100 / (len(comments_df)), 2)) + '%'
perc_neg = str(np.round(comments_df['review'].value_counts()[1] * 100 / (len(comments_df)), 2)) + '%'
perc_nut = str(np.round(comments_df['review'].value_counts()[2] * 100 / (len(comments_df)), 2)) + '%'

# Pie chart showing the share of positive and negative reviews in percentage

fig = px.pie(review_counts, values=review_counts.values, names=review_counts.index, hole=0.3,
             color_discrete_sequence=px.colors.qualitative.D3)
fig.update_traces(textposition='inside', 
                  text = (perc_pos, perc_neg, perc_nut) ,
                  textinfo='label+text', rotation=98)
fig.update_layout(title_text='Review of Udemy Courses', title_x=0.5, showlegend=False)
fig.show()

The above pie chart shows the share of each review class for the `comments_df` dataframe. Nearly 90% of the reviews are `positive` whereas 5% of the reviews falls under `negative` class. The remaining 4% reviews are `neutral`.

In [ ]:
# Reading languages_code.txt
# The text file contains a tuple list of language codes (as of ISO 639-1)

with open ('/kaggle/input/udemy-courses-additional-files/languages_code.txt') as f:
    data = f.read()
    
languages_code = ast.literal_eval(data)

# Creating dictionary of language codes
dict_languages = dict(languages_code)

# There are two versions of chinese language: 
# 'zh-cn' - Traditional chinese and 'zh-tw' - Chinese (Taiwan)
# We will give a common 'chinese' code (zh) for both of these languages
mask = (comments_df['comment_language'] == 'zh-cn') | (comments_df['comment_language'] == 'zh-tw')
comments_df.loc[mask, 'comment_language'] = 'zh'

# Transforming language codes into actual language names for readability
comments_df['comment_language'] = comments_df['comment_language'].map(dict_languages)
comments_df.head()

In [ ]:
languages_count = comments_df['comment_language'].value_counts()

fig = px.pie(languages_count, values=languages_count.values, names=languages_count.index, hole=0.25)
fig.update_traces(textposition='inside',
                 textinfo='label+percent', rotation=150)
fig.update_layout(title_text='Share of Languages in Comments', title_x=0.46, showlegend=False)
fig.show()

The data analysis of the `comment_language` column reveals that there are 52 languages in which the users commented. Out of 52 languages, English was the widely used language for commenting about the quality of the courses followed by Spanish (9.67\%) and Portuguese (8.92\%). There are a few comments that are misclassified due to insufficient word length and a mix of some languages (as shown below).

In [ ]:
# Here is an example of a comment written in both English and Tamil.
comments_df.iloc[2767]['comment']

## 4.4 Year-wise Comment Analysis

In [ ]:
# Cleaning the date column and keeping only the year, month and day values.
comments_df['date'] = comments_df['date'].str[0:10]

# Converting the object datatype into datetime datatype
comments_df['date'] = pd.to_datetime(comments_df['date'])

# Creating new columns for year and month based on date column
comments_df['year'] = comments_df['date'].dt.year
comments_df['month'] = comments_df['date'].dt.month_name()

comments_df.head()

In [ ]:
# year-wise arranging comments
year_comments = comments_df.groupby('year')['comment'].count()

# Barplot to show the number of comments per year
fig, ax = plt.subplots(figsize=(12, 7), dpi=100)
sns.barplot(x=year_comments.index, y=year_comments.values, palette='magma')
ax.yaxis.set_major_formatter(ticker.EngFormatter())
ax.set_ylabel("Number of comments")
ax.set_xlabel("Year")
ax.set_title("Number of comments posted per year");

The above bar plot displays the total comments made every year. Udemy was started in the year 2010. During the initial years up to 2016, less than 500k comments were posted. Since then, the number of comments started to go up and reached a maximum of nearly 2.5 million comments in 2020. Further, the comments posted per month for the year 2020 are also studied (shown below).

In [ ]:
# Month-wise breakup of comments for the year 2020
year_2020 = comments_df[comments_df['year'] == 2020]
year_2020_month = year_2020.groupby('month')['comment'].count()

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Barplot to visualize the comments distribution for different months in 2020
fig, ax = plt.subplots(figsize=(12, 7), dpi=100)
sns.countplot(data=year_2020, x=year_2020['month'], palette='magma', order=month_order)
ax.yaxis.set_major_formatter(ticker.EngFormatter())
ax.set_ylabel("Number of comments")
ax.set_xlabel("Month")
ax.set_title("Number of comments posted in the year 2020");

# 5. Conclusion

The two datasets related to Udemy courses and comments are investigated with the help of data analysis tools. For both datasets, the relationship between features is analyzed, and created new columns from the existing features. Following is the summary of the analysis performed:
    
1. Nearly 20% of the courses are priced at $\$$19.99. Of the total courses, around 10\% of the courses are free of charge.

2. Courses are offered in 79 languages and 59\% of the courses are taught in English.

3. The 'development' category has the highest number of courses and subscribers.

4. From the comments dataset analysis, about 91\% of the comments are positive about the courses, 5\% of the comments are negative and the remaining comments are neutral.

5. English is the most preferred language for posting comments. It is followed by Spanish and Portuguese.

6. The year 2020 saw the maximum number of comments posted on Udemy.


# 6. References

1. [Language detection library](https://pypi.org/project/langdetect/)
2. [Python Tuple Lists with language codes](https://gist.github.com/alexanderjulo/4073388)
3. [Udemy Courses Datasets](https://www.kaggle.com/datasets/hossaingh/udemy-courses)